# Single Neuron Biophysics notebooks

*David Sterratt, October 2025*

These notebooks accompany the MSc Neuroscience lecture on Single Neuron Biophysics

## Part 2: Using the NEURON simulation library

The code in Part 1 works for the very simple example, and could be extended further. However, it is also not very efficient, and specifying more complex models would require a lot of code. Modelling environments like the NEURON simulation library are designed to be very efficient, and to make model specification (fairly) intuitive.

We'll start by installing the package. Press Shift+Return in the cell below, and wait for it finish, when it will say "Successfully installed...".

In [ ]:
!pip install neuron

Now import the relevant parts of NEURON. Don't worry about the warning about the DISPLAY variable.

In [ ]:
from neuron import n
from neuron.units import ms, mV
n.load_file("stdrun.hoc")

We'll also import the graphics plotting package before we start

In [ ]:
from bokeh.io import output_notebook # A module that we need to plot 
import bokeh.plotting as plt         # A module that we need to plot 
output_notebook()

### 2.A Create and run a cell model

We will first create the simplest possible model of a patch of
membrane: a single passive compartment.

![img](schematic-cir-RC.png)

The equivalent electrical
circuit comprises a membrane capacitance $C_\text{m}$, a membrane
resistance $R_\text{m}$, a battery whose electromotive force is the same as
the resting membrane potential $V_\text{m}$ and a current source $I_\text{e}$.  We
will imagine our circuit represents a soma, though in fact it misses
some crucial features of a real soma, namely active properties.

First we'll create a soma and print its default priorities:

In [ ]:
# Create the soma
soma = n.Section("soma")
soma.psection()

We don't need to know what all of these are, but some are:
- `cm` - specific capacitance in µF/cm<sup>-2</sup>
- `L` - length of the "section" in µm, which is a cylinder
- `diam` - diameter of the "section" in µm

Note there is no membrane resistance mentioned; we will introduce this by inserting a "passive" mechanism.  NEURON has a special channel type called `pas` to
simulate the membrane resistance and reversal potential. To insert it into the membrane, type:

In [ ]:
# Insert passive channels
soma.insert(n.pas)
soma.psection()

Now we can see a `pas` in `density_mechs`. It has:
- `g` specific membrane conductance $g_\text{m}$ in S/cm<sup>2</sup>. As conductance is the inverse of resistance, the value 0.001S/cm<sup>2</sup> is equivalent to a specific membrane resistance of 1000Ωcm<sup>2</sup>
- `e` the passive associated reversal potential $E_\text{m}$ in mV
- `i` the current presently flowing though the passive mechanism in mA/cm<sup>2</sup>

We can now run the simulation with the code below. What would you expect to see?

In [ ]:
# Set up recordings of the membrane potential and time
v = n.Vector().record(soma(0.5)._ref_v)      # Membrane potential vector
i_pas = n.Vector().record(soma(0.5)._ref_i_pas)  # Current through passive mechanism
i_cap = n.Vector().record(soma(0.5)._ref_i_cap)  # Current through capacitive mechanism
t = n.Vector().record(n._ref_t)              # Time stamp vector

# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 40 milliseconds
n.continuerun(40 * ms)

In [ ]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)


### 2.C Setting the specific membrane resistance

We should see the same sort of behaviour as in the simulation we wrote earlier.

A more typical value for the specific membrane resistance is $R_\text{m} = 10$Ωcm<sup>2</sup>. This means that the specific
membrane conductance should be 0.0001 S/cm<sup>2</sup>. Run the cell below to change the value of the membrane conductance and re-run the simulation. 

In [ ]:
# Set the membrane conductance to 0.0001 S/cm2
soma(0.5).pas.g = 0.0001

# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 40 milliseconds
n.continuerun(40 * ms)

# Plot
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)

### 2.C Injecting current into a single compartment

We have now built a single compartment with passive channels. To test its properties, we will inject some current. To do this, run this cell:

In [ ]:
# Insert a current clamp electrode in the centre of the soma
stim = n.IClamp(soma(0.5))
stim.amp   =   1 # nA
stim.delay = 100 # ms
stim.dur   = 100 # ms 

This will inject a pulse of current with an amplitude of 1 nA (`stim.amp`) starting 100 ms (`stim.delay`) after the simulation has
started and lasting for 100 ms (`stim.dur`).  We will want to run the simulation for longer, to see the full effect.

In [ ]:
# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 250 milliseconds
n.continuerun(250 * ms)

f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)


We can also examine the current flowing through the membrane (blue) and the current flowing onto the membrane (red). Remember that positive current is outwards. 

In [ ]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="i (mA)", aspect_ratio=2)
f.line(t, i_pas, line_width=2)
f.line(t, i_cap, line_width=2, color='red')

plt.show(f)

Try to explain what you're seeing to yourself. Then look at the explanation below.

### Explanation

The voltage first settles to −70 mV (the resting membrane potential) from its starting value of −65 mV. Then, at 100 ms (the time of the current
injection onset), it starts to rise, though not instantaneously. This is because when current is injected it will first mainly charge the membrane capacitance, but as this charges up, more of it will flow through
the membrane resistance. It levels out at about −63.65 mV. In this steady state, all the current flows
through the membrane resistance rather than onto the membrane capacitance.
At 200 ms (the time of the current offset) the membrane potential starts to decay back to the resting
potential. This is due to the capacitance discharging slowly through the resistance. This is a bit like a
battery going flat, though on a much faster timescale!
We can check that the simulation is doing what it should be by making measurements of the simulated
results and then comparing them to the theoretical values of the same measurements.


#### The input resistance
The input resistance $R_\text{in}$ is a measure of how much current needs to be injected to increase the membrane
potential by a certain amount, and is often measured experimentally. The input resistance is defined

$$R_{\text{in}} = \frac{\Delta V}{\Delta i_\text{e}}$$

where $\Delta V$ is a small change in membrane potential due to a small change in injection current $\Delta i_{\text{e}}$. In
the case of voltage-dependent conductances, the input resistance may vary depending on the holding
membrane potential it is measured at. In our case, the conductances are fixed, so the input resistance
will not depend on the holding membrane potential and we do not need to make our change in current
and membrane potential small.